In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Werner_Syndrome/GSE48761'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of skin fibroblast and iPSC from Werner Syndrome"
!Series_summary	"The premature aging disorder Werner Syndrome (WS) is characterized by early onset of aging phenotypes resembling natural aging. In most WS patients there are mutations in the DNA helicase WRN, an enzyme important in maintaining genome stability and telomere replication. Interestingly, its clinical manifestations reflect a severe degree of deterioration for connective tissue, whereas the central nervous system is less affected. We suggest that the varied vulnerability to aging is regulated by an unknown mechanism that protects specific lineages of stem cells from premature senescence. To address this problem, we reprogrammed patient skin fibroblasts to induced pluripotent stem cells (iPSC). The expression profile for the differentiated normal and WS fibroblasts and undifferentiated iPSC were compared. A distinct expression profile was found between normal and WS

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Example of Sample Characteristics Dictionary (Reintroduce it properly from the context where it's first provided)
sample_characteristics = {
    0: ['gender: Male', 'gender: Female'],
    1: ['age: 10', 'age: 16', 'age: 7', 'age: 29', 'age: 20', 'age: 60', 'age: 30', 'age: 25', 'age: 37', 'age: 36', 'age: embryonic'],
    2: ['genotype: WT', 'genotype: WRN mutant'],
    3: ['cell type: Normal Fibroblast', 'cell type: Werner Syndrome Fibroblast', 'cell type: Normal iPSC', 'cell type: Werner Syndrome iPSC', 'cell type: Werner Syndrome iPSC, p53 knockdown', 'cell type: Werner Syndrome iPSC, hTERT expressed', 'cell type: Normal hESC']
}

def convert_trait(value):
    try:
        trait_value = value.split(":")[1].strip()
        return 1 if trait_value == "Werner Syndrome Fibroblast" else 0
    except Exception:
        return None
    
def convert_age(value):
    try:
        age_value = value.split(":")[1].strip()
        if age_value == "embryonic":
            return None
        return float(age_value)
    except Exception:
        return None

def convert_gender(value):
    try:
        gender_value = value.split(":")[1].strip()
        return 1 if gender_value.lower() == "male" else 0
    except Exception:
        return None

# Check gene expression data availability
is_gene_available = True

# Assign keys for data availability
trait_row = 3 if any("Werner Syndrome Fibroblast" in s for s in sample_characteristics[3]) else None
age_row = 1 if "age" in sample_characteristics[1][0] else None
gender_row = 0 if "gender" in sample_characteristics[0][0] else None

# Save cohort information
save_cohort_info('GSE48761', './preprocessed/Werner_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Werner_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Werner_Syndrome/trait_data/GSE48761.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1184254': [0.0, 10.0, 1.0], 'GSM1184255': [0.0, 10.0, 1.0], 'GSM1184256': [0.0, 16.0, 1.0], 'GSM1184257': [0.0, 16.0, 1.0], 'GSM1184258': [0.0, 7.0, 1.0], 'GSM1184259': [0.0, 7.0, 1.0], 'GSM1184260': [0.0, 29.0, 1.0], 'GSM1184261': [0.0, 29.0, 1.0], 'GSM1184262': [0.0, 20.0, 1.0], 'GSM1184263': [0.0, 20.0, 1.0], 'GSM1184264': [1.0, 60.0, 1.0], 'GSM1184265': [1.0, 60.0, 1.0], 'GSM1184266': [1.0, 30.0, 1.0], 'GSM1184267': [1.0, 30.0, 1.0], 'GSM1184268': [1.0, 25.0, 1.0], 'GSM1184269': [1.0, 25.0, 1.0], 'GSM1184270': [1.0, 37.0, 1.0], 'GSM1184271': [1.0, 37.0, 1.0], 'GSM1184272': [1.0, 36.0, 1.0], 'GSM1184273': [1.0, 36.0, 1.0], 'GSM1184274': [0.0, 10.0, 1.0], 'GSM1184275': [0.0, 10.0, 1.0], 'GSM1184276': [0.0, 16.0, 1.0], 'GSM1184277': [0.0, 16.0, 1.0], 'GSM1184278': [0.0, 7.0, 1.0], 'GSM1184279': [0.0, 7.0, 1.0], 'GSM1184280': [0.0, 29.0, 1.0], 'GSM1184281': [0.0, 29.0, 1.0], 'GSM1184282': [0.0, 20.0, 1.0], 'GSM1184283': [0.0, 20.0, 1.0], 'GSM1184284': [0.0, 25.0, 1.0], 'GSM1184285

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which keys store the probe IDs and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)
